In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 11 18:06:00 2022

@author: Beau.Uriona
"""

import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
from requests import Session

# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.conversions import query_to_pandas
from snowexsql.data import SiteData, PointData, LayerData, ImageData

BUFFER_IN_METERS = 10 * 1000 # 10km
SNOW_EX_GEOJSON_FILENAME = "snow_ex_all_stations.geojson"
REFRESH_SNOW_EX_GEOJSON = False
SNOWEX_DB_NAME = 'snow:hackweek@db.snowexdata.org/snowex'

if REFRESH_SNOW_EX_GEOJSON or not os.path.isfile(SNOW_EX_GEOJSON_FILENAME):
    # Get the snow ex sites
    engine, session = get_db(SNOWEX_DB_NAME)
    qry = session.query(SiteData.site_name, SiteData.site_id, SiteData.latitude, SiteData.longitude)

    df_snow_ex = query_to_pandas(qry, engine).drop_duplicates(subset=["site_id"])
    gdf_snow_ex = gpd.GeoDataFrame(
        df_snow_ex, 
        geometry=gpd.points_from_xy(df_snow_ex.longitude, df_snow_ex.latitude),
        crs=4326
    )
    gdf_snow_ex.to_file(SNOW_EX_GEOJSON_FILENAME, driver='GeoJSON')
    #gdf_snow_ex.rename(columns={"geom": "geometry"}, inplace=True)
    session.close()
else:
    gdf_snow_ex = gpd.read_file(SNOW_EX_GEOJSON_FILENAME)
    
print(f"The CRS of the Snow Ex metadata is - {gdf_snow_ex.crs}")
gdf_snow_ex_buffer = gdf_snow_ex.to_crs(26912).buffer(BUFFER_IN_METERS).to_crs(4326)
snow_ex_bounds = gdf_snow_ex.to_crs(4326).total_bounds
print(f"The bounding box of the snow Ex sites is - {snow_ex_bounds}")
snow_ex_map = gdf_snow_ex_buffer.to_crs(4326).explore()
snow_ex_map = gdf_snow_ex.to_crs(4326).explore(m=snow_ex_map)


# Get SNOTEL Sites
AWDB_API_DOMAIN = "https://api.snowdata.info"
sntl_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=SNTL&format=geojson"
msnt_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=MSNT&format=geojson"
# Get the data and rename everything to match the snow ex columns
gdf_sntl = gpd.read_file(sntl_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_msnt = gpd.read_file(msnt_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_sntl = pd.concat([gdf_msnt, gdf_sntl])
gdf_sntl = gdf_sntl[gdf_sntl["beginDate"].dt.year <= 2017]
print(f"The CRS of the Snotel metadata is - {gdf_sntl.crs}, better change it")
gdf_sntl = gdf_sntl.to_crs(gdf_snow_ex.crs)
print(f"The CRS of the Snotel metadata is now - {gdf_sntl.crs}, all good!")
# Only keep the columns we care about... for now
gdf_sntl = gdf_sntl.drop(
    columns=[i for i in gdf_sntl.columns if i not in ["geometry", "site_id", "site_name"]]
).set_geometry("geometry")
gdf_sntl_clipped = gdf_sntl.clip(gdf_snow_ex_buffer)
sntl_map = gdf_sntl_clipped.explore()
snow_ex_map = gdf_sntl_clipped.explore(m=snow_ex_map)
print(f"{len(gdf_sntl_clipped)} AWDB sites were found within {BUFFER_IN_METERS / 1000:.0f} km of SnowEx Sites")
_ = [print(f'* {row["site_name"]} ({row["site_id"]})') for idx, row in gdf_sntl_clipped.sort_values("site_name").iterrows()]
valid_triplets = list(set(gdf_sntl_clipped["site_id"]))

The CRS of the Snow Ex metadata is - epsg:4326
The bounding box of the snow Ex sites is - [-120.29898   35.85793 -105.54616   44.30447]
The CRS of the Snotel metadata is - epsg:4979, better change it
The CRS of the Snotel metadata is now - epsg:4326, all good!
28 AWDB sites were found within 10 km of SnowEx Sites
* Banner Summit (312:ID:SNTL)
* Bogus Basin (978:ID:SNTL)
* Brighton (366:UT:SNTL)
* Butte (380:CO:SNTL)
* Caples Lake (CAP:CA:MSNT)
* Carson Pass (1067:CA:SNTL)
* Devils Postpile (DPO:CA:MSNT)
* Fool Creek (1186:CO:SNTL)
* Forestdale Creek (1049:CA:SNTL)
* Idarado (538:CO:SNTL)
* Independence Camp (539:CA:SNTL)
* Independence Creek (540:CA:SNTL)
* Independence Lake (541:CA:SNTL)
* Joe Wright (551:CO:SNTL)
* Mammoth Pass (MHP:CA:MSNT)
* Mesa Lakes (622:CO:SNTL)
* Mill-D North (628:UT:SNTL)
* Mineral Creek (629:CO:SNTL)
* Mores Creek Summit (637:ID:SNTL)
* Niwot (663:CO:SNTL)
* Red Mountain Pass (713:CO:SNTL)
* Sawtooth (1251:CO:SNTL)
* Schneiders (SCN:CA:MSNT)
* Schofield Pass

In [2]:
from datetime import datetime

import requests
import pandas as pd
import numpy as np

API_DOMAIN = "https://api.snowdata.info/"

def get_awdb_data(site_ids, element="WTEQ", sdate=datetime(1899,10,1), edate=datetime.now(), orient="records", server=API_DOMAIN, sesh=None):
    """
    Takes a list of site ids or a single site id and by default returns SWE period of record data as a single or list of dataframes,
    but user can pass args to modify data returned.
    
    Valid elements include WTEQ, SNWD, PREC, SMS, STO, TAVG
    site_id takes the form of a triplet made from <network_site_id>:<state_abbrv>:<network> where network is either SNTL or MNST
    """
    dfs = []
    return_single = False
    if not isinstance(site_ids, list):
        site_ids = [site_ids]
        return_single = True
    for site_id in site_ids:
        endpoint = "data/getDaily"
        date_args = f"sDate={sdate:%Y-%m-%d}&eDate={edate:%Y-%m-%d}"
        frmt_args = f"format=json&orient={orient}"
        all_args = f"?triplet={site_id}&{date_args}&element={element}&{frmt_args}"
        url = f"{server}{endpoint}{all_args}"
        #print(f"getting data for {site_id} {element} starting {sdate:%Y-%m-%d} and ending {edate:%Y-%m-%d}")
        data_col_lbl = f"{site_id}" + ":" + f"{element}"
        if sesh:
            req = sesh.get(url)
        else:
            req = requests.get(url)
        if req.ok:
            df = pd.DataFrame.from_dict(req.json())
            df.columns = ["Date", data_col_lbl]
            df.set_index("Date", inplace=True)
        else:
            print("  No data returned!")
            df = pd.DataFrame(
                data=[
                    {"Date": pd.NaT, data_col_lbl: np.nan}
                ],
            ).set_index("Date").dropna()
        dfs.append(df)
    if return_single:
        return dfs[0]
    return dfs

with requests.Session() as sesh:
    dfs_swe = get_awdb_data(site_ids=valid_triplets, element="WTEQ", sdate=datetime(2016, 10, 1), sesh=sesh)
    dfs_snwd = get_awdb_data(site_ids=valid_triplets, element="SNWD", sdate=datetime(2016, 10, 1), sesh=sesh)
df_all_swe_data = pd.concat(dfs_swe, axis=1)
df_all_snwd_data = pd.concat(dfs_snwd, axis=1)
df_all_awdb_data = pd.concat([df_all_swe_data, df_all_snwd_data], axis=1).fillna(np.nan)
df_all_awdb_data.index = pd.to_datetime(df_all_awdb_data.index)
df_all_awdb_data.tail(2)


getting data for 380:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 629:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 366:UT:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 1251:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 814:UT:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 541:CA:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 312:ID:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 1049:CA:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 737:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 622:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 838:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 663:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 538:CO:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting da

,380:CO:SNTL:WTEQ,629:CO:SNTL:WTEQ,366:UT:SNTL:WTEQ,1251:CO:SNTL:WTEQ,814:UT:SNTL:WTEQ,541:CA:SNTL:WTEQ,312:ID:SNTL:WTEQ,1049:CA:SNTL:WTEQ,737:CO:SNTL:WTEQ,622:CO:SNTL:WTEQ,...,637:ID:SNTL:SNWD,766:UT:SNTL:SNWD,539:CA:SNTL:SNWD,540:CA:SNTL:SNWD,MHP:CA:MSNT:SNWD,SIL:CA:MSNT:SNWD,1186:CO:SNTL:SNWD,713:CO:SNTL:SNWD,628:UT:SNTL:SNWD,SCN:CA:MSNT:SNWD
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-13,0.0,0.1,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN
2022-07-14,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,0.0,9.0,0.0,NaN


In [3]:
engine, session = get_db(SNOWEX_DB_NAME)
use_cols = ("site_id", "date", "pit_id", "value", "geometry", "depth", "instrument")
obj = {}
data_dfs = []
for awdb_site_id in valid_triplets[4:]:
    gdf_awdb_site_buffer = gdf_sntl[gdf_sntl["site_id"] == awdb_site_id].to_crs(26912).buffer(BUFFER_IN_METERS).to_crs(4326)
    gdf_snow_ex_inside = gdf_snow_ex.clip(gdf_awdb_site_buffer)
    snow_ex_site_list = gdf_snow_ex_inside["site_id"].tolist()
    awdb_site_name = gdf_sntl[gdf_sntl["site_id"] == awdb_site_id]["site_name"].values[0]
    print(
        f"The AWDB site: {awdb_site_name} has {len(snow_ex_site_list)} SnowEx pits within "
        f"{BUFFER_IN_METERS / 1000:.0f} km ({', '.join(snow_ex_site_list)})."
    )
    
    qry = session.query(LayerData).filter(LayerData.site_id.in_(snow_ex_site_list)).filter(LayerData.type == "density")
    df_snow_ex_data = query_to_pandas(qry, engine)
    gdf_snow_ex_data = gpd.GeoDataFrame(
        df_snow_ex_data, 
        geometry=gpd.points_from_xy(df_snow_ex_data.longitude, df_snow_ex_data.latitude),
        crs=4326
    )
    gdf_snow_ex_data.drop(columns=[i for i in df_snow_ex_data.columns if i not in use_cols], inplace=True)
    gdf_snow_ex_data.loc[:, "value"] = gdf_snow_ex_data["value"].astype(float)
    gdf_snow_ex_data.index = pd.to_datetime(gdf_snow_ex_data["date"])
    df_grp_by = gdf_snow_ex_data.groupby(by=[gdf_snow_ex_data.index, "pit_id"])
    df_date = df_grp_by["date"].first()
    df_bulk_density = df_grp_by["value"].mean() / 1000 # convert kg/m^3
    df_depth = df_grp_by["depth"].max() * 10 # convert cm to mm
    df_swe = df_depth * df_bulk_density
    df_values = gpd.GeoDataFrame(data={"swe_snowex": df_swe.round(0), "depth_snowex": df_depth, "date": df_date})#.set_geometry("geometry")
    df_values.index = pd.to_datetime(df_values["date"])
    gdf_snow_ex_data.drop(columns="date", inplace=True)
    df_pit_data = df_grp_by.first().drop(columns=["depth", "value"])
    df_pit_data = df_pit_data.reset_index(level='pit_id')
    df_combined = df_pit_data.join(df_values).rename(columns={"site_id": "snowex_id", "pit_id": "snowex_pit_id"})
    df_awdb_site = df_all_awdb_data[[f"{awdb_site_id}:WTEQ", f"{awdb_site_id}:SNWD"]]
    df_combined = df_combined.join((df_awdb_site / 0.0393701).round(0)).rename(columns={f"{awdb_site_id}:WTEQ": "swe_awdb", f"{awdb_site_id}:SNWD": "depth_awdb"})
    df_combined["awdb_id"] = awdb_site_id
    df_combined.set_crs(4326, inplace=True)
    sntl_point = gdf_sntl_clipped[gdf_sntl_clipped["site_id"] == awdb_site_id]["geometry"].to_crs(26912).values[0]
    df_combined["distance"] = df_combined.to_crs(26912).distance(sntl_point).round(0)
    data_dfs.append(df_combined)

obj["data"] = pd.concat(data_dfs).to_crs(4326)
obj["meta"] = {
    "awdb": gdf_sntl_clipped.to_crs(4326),
    "snowex": gdf_snow_ex.to_crs(4326)
}

The AWDB site: Thaynes Canyon has 2 SnowEx pits within 10 km (Alta Collins, Atwater).
The AWDB site: Independence Lake has 3 SnowEx pits within 10 km (Tower 4, Forest, Open).
The AWDB site: Banner Summit has 2 SnowEx pits within 10 km (Banner Snotel, Banner Open).
The AWDB site: Forestdale Creek has 1 SnowEx pits within 10 km (Caples Lake).
The AWDB site: Schofield Pass has 1 SnowEx pits within 10 km (Gothic).
The AWDB site: Mesa Lakes has 48 SnowEx pits within 10 km (6C37, 7S50, 3S47, 2S48, 2S45, 5S42, 6S53, 2S46, 5S49, 3S52, 8S41, 6S44, 9S51, 5S43, 8N55, 3N53, 5N50, 7N57, 9N59, 2N49, 8N58, County Line Open, 8N54, 8N52, 8N45, 9N56, 8N51, 9N47, County Line Tree, 2N48, GML, Skyway Open, Skyway Tree, 6C34, 3S38, 9S40, 9S39, 2S37, 6N46, 9N44, 5N41, 8N38, 7N40, 9N39, 9N43, 8N37, 6N36, 9N42).
The AWDB site: University Camp has 6 SnowEx pits within 10 km (Forest Flat, Forest North, C1, Open Flat, Forest South, Saddle).
The AWDB site: Niwot has 6 SnowEx pits within 10 km (Forest Flat, Forest 

In [4]:
import pickle
import json

with open("one_obj_to_rule_them_all.pkl", "wb") as p:
    pickle.dump(obj, p)

with open("one_obj_to_rule_them_all.json", "w") as j:
    json.dump(
        {
            "data": json.loads(pd.concat(data_dfs).to_crs(4326).astype({"date": str}).to_json()),
            "meta": {
                "awdb": json.loads(gdf_sntl_clipped.to_crs(4326).to_json()),
                "snowex": json.loads(gdf_snow_ex.to_crs(4326).to_json())
            }
        },
        j,
        indent=4
    )

In [5]:
obj["data"].sort_values("distance").head(30)

,snowex_pit_id,instrument,snowex_id,geometry,swe_snowex,depth_snowex,date,swe_awdb,depth_awdb,awdb_id,distance
2020-03-05,IDBRBS_20200305_1040,None,Banner Snotel,POINT (-115.23458 44.30358),368.0,1150.0,2020-03-05,422.0,1245.0,312:ID:SNTL,20.0
2020-03-05,IDBRBS_20200305_1040,None,Banner Snotel,POINT (-115.23458 44.30358),414.0,1260.0,2020-03-05,422.0,1245.0,312:ID:SNTL,20.0
2020-02-19,IDBRBS_20200219_1045,None,Banner Snotel,POINT (-115.23454 44.30360),363.0,1320.0,2020-02-19,417.0,1422.0,312:ID:SNTL,21.0
2020-02-19,IDBRBS_20200219_1045,None,Banner Snotel,POINT (-115.23454 44.30360),390.0,1400.0,2020-02-19,417.0,1422.0,312:ID:SNTL,21.0
2020-02-27,IDBRBS_20200227_1030,None,Banner Snotel,POINT (-115.23464 44.30358),401.0,1330.0,2020-02-27,422.0,1346.0,312:ID:SNTL,22.0
2020-02-27,IDBRBS_20200227_1030,None,Banner Snotel,POINT (-115.23464 44.30358),366.0,1240.0,2020-02-27,422.0,1346.0,312:ID:SNTL,22.0
2020-01-30,IDBRBS_20200130_1200,None,Banner Snotel,POINT (-115.23462 44.30359),311.0,1310.0,2020-01-30,356.0,1422.0,312:ID:SNTL,22.0
2020-01-30,IDBRBS_20200130_1200,None,Banner Snotel,POINT (-115.23462 44.30359),300.0,1270.0,2020-01-30,356.0,1422.0,312:ID:SNTL,22.0
2020-03-12,IDBRBS_20200312_1000,None,Banner Snotel,POINT (-115.23460 44.30359),382.0,1180.0,2020-03-12,439.0,1295.0,312:ID:SNTL,22.0
2020-03-12,IDBRBS_20200312_1000,None,Banner Snotel,POINT (-115.23460 44.30359),429.0,1330.0,2020-03-12,439.0,1295.0,312:ID:SNTL,22.0
